In [16]:
import seaborn as sns
import pandas as pd
import missingno as msno
import numpy as np
from sklearn.impute import SimpleImputer
import difflib
from random import sample
import folium
import math
import matplotlib.pyplot as plt
import seaborn as sns
from folium.plugins import HeatMap
from folium.features import CustomIcon
from folium.plugins import FastMarkerCluster, MarkerCluster
import json
import pyreadstat
import string

In [2]:
postcodecsv = pd.read_csv('au_postcodes.csv') 
# References: 
#https://blog.greglow.com/2019/11/05/power-bi-creating-a-topojson-file-of-australian-postcodes-for-use-with-shape-map/
#https://mapshaper.org/ 
#https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.003July%202016?OpenDocument

import pandas as pd
from statistics import median
from statistics import mean
from scipy import stats

ndf = pd.read_json('POA_2016_AUST.json')
ndf1 = ndf['features'].apply(pd.Series)
ndf2 = ndf1['geometry'].apply(pd.Series)
ndf3 = ndf1['properties'].apply(pd.Series)
border = ndf2['coordinates']

depth = lambda L: isinstance(L, list) and max(map(depth, L))+1

def flatten(d, l=1):
    for i in d:
        yield from ([i] if l == 1 else flatten(i, l-1))

def centerlocation(index):
    tmplst = list(flatten(border[index], l= depth(border[index])))
    longi = median([i for i in tmplst  if i > 0])
    lati = median([i for i in tmplst  if i < 0])
    return[longi,lati]

postcodedf = []
i = 0
for i in range(len(ndf)):
    if depth(border[i])>0:
        tpd = {
        'postcode':[ndf3['POA_CODE16'].iloc[i]],
        'longitude':[centerlocation(i)[0]],
        'latitude':[centerlocation(i)[1]]}
        tpdf = pd.DataFrame(tpd)
        postcodedf.append(tpdf)
    else:
        print(i)
        continue
    i = i+1        
postcodedf = pd.concat(postcodedf,ignore_index=True)

2668
2669


In [4]:
data =  pd.read_csv('./markettrends0.csv', dtype = {'state': 'str', 'sa3_name16': 'str','sa4_name16': 'str',
                                                   'postcode': 'str','state': 'str','property_type':'str'})
ndata = data.fillna({'Volume of new rental listings (1 month)': 0, 'Volume of sales (1 month)': 0}).dropna(subset = ['postcode'])
ndatahouses = ndata[:][ndata.property_type == 'Houses']
ndataunits = ndata[:][ndata.property_type == 'Units']

In [5]:
# So far, Houses only
UniqueNames_sa3 = ndatahouses.sa3_name16.unique()[:-1]
DataFrameDict_sa3 = {elem : pd.DataFrame for elem in UniqueNames_sa3}
for key in DataFrameDict_sa3.keys():
     DataFrameDict_sa3[key] = ndata[:][ndata.sa3_name16 == key]

In [17]:
year = list(range(2001,2021))
DataFrameDict_HILDA = {elem : pd.DataFrame for elem in year}
for key in DataFrameDict_HILDA.keys():
    order = key - 2001
    ini = string.ascii_lowercase[order]
    df, meta = pyreadstat.read_dta(f'Household_{ini}200u.dta')
    df.columns = df.columns.str[1:]
    DataFrameDict_HILDA[key] = df
list_postcodes = DataFrameDict_HILDA[2001]['hhpcode']
for key in DataFrameDict_HILDA.keys():
    list_ny = DataFrameDict_HILDA[key]['hhpcode']
    list_postcodes =list(set(list_postcodes).intersection(list_ny))
count_dict = dict()
for key in DataFrameDict_HILDA.keys():
    for item in DataFrameDict_HILDA[key]['hhpcode'].unique():
        if item in count_dict:
            count_dict[item] +=1
        else:
            count_dict[item] =1
df_allpostcode = pd.DataFrame(count_dict.items()) 

# if the amount of missing values are greater than 5, delete!



In [19]:
df_retainedpc = df_allpostcode.drop(df_allpostcode[df_allpostcode[1] < 16].index)

In [21]:
df_retainedpc[0]

0       2018
1       2039
2       2037
3       2048
4       2044
        ... 
1380    3820
1382    4883
1386    3464
1389    3043
1391    3685
Name: 0, Length: 1052, dtype: object

In [22]:
UniquePostcode = df_retainedpc[0]
DataFrameDict_postcode = {elem : pd.DataFrame for elem in UniquePostcode}
for key in DataFrameDict_postcode.keys():
    DataFrameDict_postcode[key] = ndatahouses[:][ndatahouses.postcode == key]

In [26]:
def unique_non_null(s):
    return s.dropna().unique()
#placepc  = postcodedf['Postcode']

growthrate = []

for key in DataFrameDict_postcode.keys():
    if  sum(postcodedf['postcode']==key)>0:
        # ind = placepc.loc[placepc == pc].index[0]
        ind = postcodedf.loc[postcodedf['postcode']== key].index[0]
        df = DataFrameDict_postcode[key]
        if len(df['Hedonic Home Value Index'])>360:
            tpd = {
                'UniqueNames_sa3':DataFrameDict_postcode[key]['sa3_name16'].unique(),
                'S/T':DataFrameDict_postcode[key]['state'].unique(),
                #'Postcode':[postcodedf.loc[ind,'Postcode']],
               # 'longitude':[postcodedf.loc[ind,'longitude']],
                #'latitude':[postcodedf.loc[ind,'latitude']],
                'Postcode':[postcodedf.loc[ind,'postcode']],
                'longitude':[postcodedf.loc[ind,'longitude']],
                'latitude':[postcodedf.loc[ind,'latitude']],
                'Growth_Houses': [df['Hedonic Home Value Index'].iloc[-1]/df['Hedonic Home Value Index'].iloc[359]], 
                'FinalHPI_Houses': [df['Hedonic Home Value Index'].iloc[-1]],
               # 'Suburbs':", ". join( ((postcodecsv.loc[(postcodecsv['postcode'] ==int(postcodedf.loc[ind,'Postcode']))]['place_name']).tolist()))}
                'Suburbs':", ". join( ((postcodecsv.loc[(postcodecsv['postcode'] == int(key))]['place_name']).tolist()))}
            tpdf = pd.DataFrame(tpd)
            growthrate.append(tpdf)
    else:
        print("area %s: data incomplete" %(key)) 
        continue  
    
growthrate = pd.concat(growthrate,ignore_index=True)
growthrate['Postcode'] = growthrate['Postcode'].astype(str)

area -7: data incomplete
area -2: data incomplete


In [24]:


ndfsa3 = pd.read_json('SA3_2016_AUST.json')
ndf1sa3 = ndfsa3['features'].apply(pd.Series)
ndf2sa3 = ndf1sa3['geometry'].apply(pd.Series)
ndf3sa3 = ndf1sa3['properties'].apply(pd.Series)
borderdsa3 = ndf2sa3['coordinates']
placename = ndf3sa3['SA3_NAME16']


growthsa3 = []

i = -1

for key in DataFrameDict_sa3.keys():
    i = i+1  
    ind = placename[placename == difflib.get_close_matches(key, placename)[0]].index[0]
    df = DataFrameDict_sa3[key]
    tpd = {'UniqueNames_sa3':[DataFrameDict_sa3[key]['sa3_name16'].unique()[0]],
            'sa3code':[ndf3sa3.loc[ind,'SA3_CODE16']], 
            'FinalHPI_Houses': [DataFrameDict_sa3[key][df.value_at_date == '2021-05-31']['Hedonic Home Value Index'].mean()]}
    tpdf = pd.DataFrame(tpd)
    growthsa3.append(tpdf)
growthsa3= pd.concat(growthsa3,ignore_index=True)  

In [29]:
DataFrameDict_sa3.keys()

dict_keys(['Darwin City', 'Darwin Suburbs', 'Litchfield', 'Palmerston', 'Daly - Tiwi - West Arnhem', 'Katherine', 'Barkly', 'Alice Springs', 'East Arnhem', 'Sydney Inner City', 'Botany', 'Eastern Suburbs - North', 'Eastern Suburbs - South', 'Leichhardt', 'Marrickville - Sydenham - Petersham', 'Strathfield - Burwood - Ashfield', 'Canada Bay', 'North Sydney - Mosman', 'Chatswood - Lane Cove', 'Ku-ring-gai', 'Hornsby', 'Warringah', 'Manly', 'Pittwater', 'Ryde - Hunters Hill', 'Carlingford', 'Pennant Hills - Epping', 'Parramatta', 'Baulkham Hills', 'Auburn', 'Merrylands - Guildford', 'Blacktown', 'Rouse Hill - McGraths Hill', 'Dural - Wisemans Ferry', 'Fairfield', 'Campbelltown (NSW)', 'Bringelly - Green Valley', 'Liverpool', 'Bankstown', 'Canterbury', 'Kogarah - Rockdale', 'Hurstville', 'Cronulla - Miranda - Caringbah', 'Sutherland - Menai - Heathcote', 'Gosford', 'Wyong', 'Lake Macquarie - West', 'Lake Macquarie - East', 'Newcastle', 'Lower Hunter', 'Great Lakes', 'Port Stephens', 'Maitl

In [34]:
folium_map4 = folium.Map(location=[-30, 138],
                        zoom_start=4.4, titles ='cartodbpositron')


f = open('POA_2016_AUST.json')
postcode_geo =json.load(f)
postcodedata2 = pd.DataFrame({'postcode':  df_retainedpc[0], 'fill': [1] * len(df_retainedpc[0])})

folium.Choropleth(
    geo_data=postcode_geo,
    name="choropleth",
    data= postcodedata2,
    fill_color="BrBG",
    columns=["postcode","fill"],
    key_on="feature.properties.POA_CODE16",
    fill_opacity=0.4,
    line_opacity=0.6
).add_to(folium_map4)

mc1 = MarkerCluster(name="Marker Cluster")
df = growthrate
for index, row in df.iterrows():
    html = " Postcode: {}<br><br>Suburb(s): {}<br><br> Location (SA3): {} <br><br>State: {} <br><br>Current HPI: {}<br><br> Fixed Based Growth Rate since 31/12/2009: {}  ".format(
                      row["Postcode"],
                      row["Suburbs"],
                      row["UniqueNames_sa3"],   
                      row["S/T"], 
                      row["FinalHPI_Houses"],
                      format(row["Growth_Houses"], '.4f'),
                     
                      )


    iframe = folium.IFrame(html)
    popup = folium.Popup(iframe,
                     min_width=400,
                     max_width=400)
    folium.CircleMarker(location=[row["latitude"],row["longitude"]],
                        color="blue",
                        fill_opacity=math.pow(row['Growth_Houses'],4)/10,
                        weight=0.8,
                        popup=popup,
                        fill=True).add_to(mc1)

mc1.add_to(folium_map4)

folium.LayerControl().add_to(folium_map4)

folium_map4





In [35]:
folium_map4.save("index.html")